# Done by Mohamad Alkadri

# Assignment 9

Build a character-level neural language model that will learn to generate new tweets. The training data consists of tweets by Donald Trump.

# Part 1: Character-level Neural Language Model

 You will build a character-level neural language model that will learn to generate new tweets.

I followed this tutorial : https://machinelearningmastery.com/develop-character-based-neural-language-model-keras/

## Import Libraries

In [1]:
'''implement your code'''
import pandas as pd
import numpy as np
import re
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Embedding, LSTM, ZeroPadding2D
import random
from keras.models import Model
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.resnet50 import preprocess_input
import matplotlib.pyplot as plt
from scipy import misc
from keras.models import model_from_json
import time 
import glob
import re
import os
import sys

'''os.mkdir("faceid_train")
os.mkdir("faceid_val")'''
import numpy
import keras
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img, img_to_array



Using TensorFlow backend.


## Load Data

In [3]:
'''implement your code'''
df=pd.read_csv("/home/cmps299/Desktop/ML/assi9/trump.csv")
df.head()

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for iPhone,RT @realDonaldTrump: The Fake News is showing ...,11-23-2018 23:49:04,45323.0,0,true,1.066116e+18
1,Twitter for iPhone,I am extremely happy and proud of the job bein...,11-23-2018 23:48:19,6132.0,24473,false,1.066116e+18
2,Twitter for iPhone,Really good Criminal Justice Reform has a true...,11-23-2018 17:14:19,11527.0,47137,false,1.066017e+18
3,Twitter for iPhone,Really good Criminal Justice Reform has a true...,11-23-2018 12:57:18,9106.0,37997,false,1.065952e+18
4,Twitter for iPhone,Republicans and Democrats MUST come together f...,11-23-2018 12:41:50,27651.0,115988,false,1.065949e+18


# Data Preparation

In [5]:
texts=df["text"].values
raw_text=""
for text in texts:
    raw_text+=" "+text
#cleaning 
raw_text=raw_text.lower().split() # to lower case
raw_text=" ".join(raw_text)
text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", raw_text)
text = re.sub(r"what's", "what is ", text)
text = re.sub(r"\'s", " ", text)
text = re.sub(r"\'ve", " have ", text)
text = re.sub(r"n't", " not ", text)
text = re.sub(r"i'm", "i am ", text)
text = re.sub(r"\'re", " are ", text)
text = re.sub(r"\'d", " would ", text)
text = re.sub(r"\'ll", " will ", text)
text = re.sub(r",", " ", text)
text = re.sub(r"\.", " ", text)
text = re.sub(r"!", " ! ", text)
text = re.sub(r"\/", " ", text)
text = re.sub(r"\^", " ^ ", text)
text = re.sub(r"\+", " + ", text)
text = re.sub(r"\-", " - ", text)
text = re.sub(r"\=", " = ", text)
text = re.sub(r"'", " ", text)
text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
text = re.sub(r":", " : ", text)
text = re.sub(r" e g ", " eg ", text)
text = re.sub(r" b g ", " bg ", text)
text = re.sub(r" u s ", " american ", text)
text = re.sub(r"\0s", "0", text)
text = re.sub(r" 9 11 ", "911", text)
text = re.sub(r"e - mail", "email", text)
text = re.sub(r"j k", "jk", text)
raw_text = re.sub(r"\s{2,}", " ", text)

#add more cleaning .....

In [6]:
# organize into sequences of characters
length = 10
sequences = list()
for i in range(length, len(raw_text)):
    # select sequence of tokens
    seq = raw_text[i-length:i+1]
    # store
    sequences.append(seq)
print('Total Sequences: %d' % len(sequences))


Total Sequences: 4302531


In [17]:
sequences[1:10]

['T @realDona',
 ' @realDonal',
 '@realDonald',
 'realDonaldT',
 'ealDonaldTr',
 'alDonaldTru',
 'lDonaldTrum',
 'DonaldTrump',
 'onaldTrump:']

# Encode Sequences


In [7]:
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))

In [9]:
encoded_sequences = list()
for line in sequences:
    # integer encode line
    encoded_seq = [mapping[char] for char in line]
    # store
    encoded_sequences.append(encoded_seq)



In [10]:
encoded_sequences[1:10]

[[36, 0, 34, 21, 17, 28, 20, 31, 30, 17, 28],
 [0, 34, 21, 17, 28, 20, 31, 30, 17, 28, 20],
 [34, 21, 17, 28, 20, 31, 30, 17, 28, 20, 36],
 [21, 17, 28, 20, 31, 30, 17, 28, 20, 36, 34],
 [17, 28, 20, 31, 30, 17, 28, 20, 36, 34, 37],
 [28, 20, 31, 30, 17, 28, 20, 36, 34, 37, 29],
 [20, 31, 30, 17, 28, 20, 36, 34, 37, 29, 32],
 [31, 30, 17, 28, 20, 36, 34, 37, 29, 32, 0],
 [30, 17, 28, 20, 36, 34, 37, 29, 32, 0, 14]]

In [11]:
# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)


Vocabulary Size: 43


# Split Inputs and Output


In [12]:
encoded_sequences = np.array(encoded_sequences)
X, y = encoded_sequences[:,:-1], encoded_sequences[:,-1]

In [36]:
X.shape

(4341839, 10)

# One_hot encoding

In [13]:
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size)

In [16]:
print(X.shape,y.shape)

(4302531, 10, 43) (4302531, 43)


## Build a Character-level Neural Language Model

You have to build an architecture for a character-level neural language model. Your model should predict the next character given a sequence of characters. Given that those are tweets, you can assume that the maximum length of a sequence is 140 characters.

In [19]:
'''implement your code'''

# define model
model = Sequential()
model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 75)                35700     
_________________________________________________________________
dense_1 (Dense)              (None, 43)                3268      
Total params: 38,968
Trainable params: 38,968
Non-trainable params: 0
_________________________________________________________________
None


## Train Model

Train your model on the dataset

In [24]:
'''implement your code'''
curr=time.time()
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, epochs=100, verbose=1)
print(time.time()/curr)

Epoch 1/100
4302531/4302531 [==============================] - 618s 144us/step - loss: 1.6398 - acc: 0.5203
Epoch 2/100
4302531/4302531 [==============================] - 600s 139us/step - loss: 1.5145 - acc: 0.5548
Epoch 3/100
4302531/4302531 [==============================] - 614s 143us/step - loss: 1.4819 - acc: 0.5637
Epoch 4/100
4302531/4302531 [==============================] - 615s 143us/step - loss: 1.4657 - acc: 0.5683
Epoch 5/100
4302531/4302531 [==============================] - 616s 143us/step - loss: 1.4550 - acc: 0.5712
Epoch 6/100
4302531/4302531 [==============================] - 616s 143us/step - loss: 1.4476 - acc: 0.5733
Epoch 7/100
4302531/4302531 [==============================] - 617s 143us/step - loss: 1.4422 - acc: 0.5746
Epoch 8/100
4302531/4302531 [==============================] - 651s 151us/step - loss: 1.4381 - acc: 0.5758
Epoch 9/100
4302531/4302531 [==============================] - 608s 141us/step - loss: 1.4346 - acc: 0.5768
Epoch 10/100
4302531/4302531

In [310]:
# save the model to file
model.save('/home/cmps299/Desktop/ML/assi9/model.h5')

In [ ]:
#possible ways to improve the accuracy: 
# 1- add more layers to lstm with more units 
# 2- Increase the sequence length from 10 till 60 
# 3- Do less data cleaning

## Generate Tweets

It's time to generate new tweets

In [123]:
df["text"][10]

'Justice Roberts can say what he wants but the 9th Circuit is a complete &amp; total disaster. It is out of control has a horrible reputation is overturned more than any Circuit in the Country 79% &amp; is used to get an almost guaranteed result. Judges must not Legislate Security...'

In [124]:
from pickle import load
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
prob_list=0
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        global prob_list
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
#         print(encoded.shape)
#         encoded = encoded.reshape(1, encoded.shape[0], encoded.shape[1])
        # predict character
    
#         yhat = model.predict_classes(encoded, verbose=0)

    #sampling; by choosing one element from the highest two predicted characters 
        prob_list = list(model.predict(encoded, verbose=0)[0])
        sorted_prob_list=np.sort(prob_list)
        prob_list=[str(elem) for elem in prob_list]
        sorted_prob_list= [str(elem) for elem in sorted_prob_list]
        top_2=sorted_prob_list[len(sorted_prob_list)-2:]
        prob=random.sample(top_2,k=1)[0]
        yhat=prob_list.index(prob)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text

# # load the model
model = load_model('/home/cmps299/Desktop/ML/assi9/model.h5')

# # test1
print(generate_seq(model, mapping, 10, 'The Fake News'.lower(), 140))
# # test 2
print("-------------------------------------------")
print(generate_seq(model, mapping, 10, 'Really good Criminal'.lower(), 140))
# # # test 3
print("-------------------------------------------")
print(generate_seq(model, mapping, 10, 'reputation is overturned'.lower(), 140))

the fake news inteldotatings to securelitivers annoust amp; sena into ameritionse amend a secrac :jinding :jreshian :jrenni too ! https : catmoment ameni
-------------------------------------------
really good criminalsing tonedov :irpbrica to todays and witch rest androokdaker to them iss than tomoor that issually todomk thank u are thanksgerting to than 
-------------------------------------------
reputation is overturneds amp;cl thank yeght :jizch truesdoy amendments todom intervia annienstingstone : tradis todomous to todoyovate they windrain : readybiaters


# Part 2: Dies Detection